<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_cassandra_issues_jira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Dependencies

In [1]:
print('Install dependencies...')
print('Update apt. ')
!sudo apt update > apt_update.log
print('Install software-properties-common.')
!sudo apt install software-properties-common > spc.log
print('Add ppa-git-core repository.')
!sudo add-apt-repository -y ppa:git-core/ppa > ppa_git_core.log
print('Install the latest version of git.')
!sudo apt install -y git > install_git.log
!git --version > git_version.log
print('Install pydriller')
!pip3 install pydriller > install_pydriller.log
!pip3 list | grep PyDriller > versao_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install dependencies...
Update apt. 


Install software-properties-common.


Add ppa-git-core repository.
Install the latest version of git.


Install pydriller
Install gitpython.
Install Jira Python lib.
All depenpencies installed!
cat: install.log: input file is output file
Details in install.log


In [2]:
from jira import JIRA
import pandas as pd
import os
import subprocess
import requests
import threading
import datetime
from git import Repo
import configparser # para manipular arquivos .conf
import pydriller

def get_repository_name(url):
  repository_name = os.path.splitext(os.path.basename(url))[0]
  return repository_name

class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        try:
          for i in range(self.qtd_repositorios):
              nome = get_repository_name(url=self.lista_urls[i])
              tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
              tdi.name = 'Thread-'+str(i+1)
              inicio_thread = datetime.datetime.now()
              print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
              self.lista_threads.append(tdi)
              elemento = (nome, inicio_thread)
              self.lista_tempo_inicio.append(elemento)
              tdi.start()

          for each in self.lista_threads:
              each.join()
        except Exception as ex:
          print(f'Erro no clona_repositorio: {str(ex)}')

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de clonagem do {t1[0]} é: {t2[1]-t1[1]} s")

# extrai o issue tracker salvo na mensagem de cada commit
def extract_cassandra_issue(text):
  """
  Extract the CASSANDRA-XYZ issue from the text.

  Args:
    text: The text to extract the issue from.

  Returns:
    The issue number, or None if the issue cannot be found.
  """

  issues_number = []

  lines = text.split('\n')
  for line in lines:
    if 'CASSANDRA-' in line :
      issue_number = line.split('-')[1]
      issue_number = 'CASSANDRA-' + issue_number
      issues_number.append(issue_number)
  return issues_number

# Commit Analysis

In [3]:
link_repositorio = 'https://github.com/apache/cassandra.git'

# Clona e repositorio
lista_repositorios = [link_repositorio]
qtd_repositorios = len(lista_repositorios)
cloner = RepoCloner(qtd_repositorios=len(lista_repositorios), lista_urls=lista_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()
print(f'Clonagem de todos os {qtd_repositorios} concluída com sucesso!')

Thread 1 iniciada em 2023-09-12 21:07:21.229627 - clonando cassandra:https://github.com/apache/cassandra.git...
Erro ao clonar o repo: https://github.com/apache/cassandra.git - Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/apache/cassandra.git cassandra
  stderr: 'fatal: destination path 'cassandra' already exists and is not an empty directory.
'
Clonagem de todos os 1 concluída com sucesso!


## Faixa de commits analisados: 3.0.0 a 3.11.11

In [4]:
# 3. Define a faixa de commits que será analisada
# 3.1 Mostra todos os commits entre as tags 3.0.0 e 3.11.11
!cd cassandra && git log --pretty="%H;%ai;%s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11-full.txt
!cd cassandra && git log --pretty="%H %s" cassandra-3.0.0...cassandra-3.11.11 > commits-3.0.0-3.11.11-msg.txt
!echo "10 primeiros commits entre as versões cassandra-3.0.0...cassandra-3.11.11"
!cd cassandra && head commits-3.0.0-3.11.11-msg.txt
!echo "..."

# Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11
!echo "Total de commits registrados entre as versões cassandra-3.0.0...cassandra-3.11.11: "
!cd cassandra && cat commits-3.0.0-3.11.11-msg.txt | wc -l

10 primeiros commits entre as versões cassandra-3.0.0...cassandra-3.11.11
4cafe2288e56e1135d65e76adbcd6c2de9306d6b Prepare for release 3.11.11
4950878639056261746e24b2a5de29d739668e2d update IDEA setup generation to mark test/conf as test resources
8857d0448914c2b8bb3e4e8f5f49b1d369f53893 Fix broken IntelliJ configuration Authored by Jacek Lewandowski; reviewed by Ekaterina Dimitrova and Michael Semb Wever for CASSANDRA-16618
3e54ed76b29b752f2c17baa98901adf47e5897d1 Don't put apidocs (javadoc) into the binary artifact
7fafda8e9063212168487ae98aad044837b7e2ab Don't fork jvms in the build
b2cfe2e5d7662c4d5900483d1838c718740ea13c Fix warning about multiple versions of ant detected when running ant test patch by Ekaterina Dimitrova; reviewed by Brandon Williams and Mick Semb Wever for CASSANDRA-16571
fd7732389aa77a94d3d5879165e8acde48125d5b Update IDE, Eclipse and NetBeans, classpaths for tests
c44dfab9b5b62820d70aba062ba3c9b9117912db Add JStackJUnitTask to avoid downloading the jar
e69505

### Lista de commits entre 3.0.0 e 3.11.11

In [5]:
# Lista os commits entre duas tags
!cd cassandra && git log --pretty="%H" cassandra-3.0.0...cassandra-3.11.11 > commmitstags300to31111.txt
commits_tag_3_from_000_to_111111 = !cd cassandra && cat commmitstags300to31111.txt
print(f'Qtd: { len(commits_tag_3_from_000_to_111111) }, {commits_tag_3_from_000_to_111111}')

In [51]:
#!cd cassandra && git show a37f8becf2d5df2895febbd8ec0cbd8ba793be8f
#!cd cassandra && git show 4cafe2288e56e1135d65e76adbcd6c2de9306d6b
# git log --format=%B -n 1 <commit>
!cd cassandra && git log --format=%B -n 1 093454205498bad7094cb8fb362d60e95296b092

Only log yaml config once, at startup

patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217



In [7]:
commit_inicial = commits_tag_3_from_000_to_111111[-1]
commit_final = commits_tag_3_from_000_to_111111[0]
data_commit_inicial = '03/08/2015'
data_commit_final = '25/06/2021'
lista_commits_analisados = commits_tag_3_from_000_to_111111

# Dicionário commit e mensagem

Obs: análise força bruta...

In [8]:
# versao forca bruta
dict_message_commits = {}
for each_commit in commits_tag_3_from_000_to_111111:
  os.environ['COMMIT'] = each_commit
  message_commit = !cd cassandra && git log --format=%B -n 1 $COMMIT
  dict_message_commits[each_commit] = message_commit

### Lista de Issue Id dos commits da faixa analisada (3.0.0 a 3.11.11)

Referente aos commits que tiveram o id do issue salvo na mensagem de commit

In [9]:
with open('/content/cassandra/commits-3.0.0-3.11.11-msg.txt', mode='r') as f:
  conteudo = f.read()

list_issue_number = extract_cassandra_issue(conteudo)

# Faz a limpeza para garantir o id do Issue tracker
list_cassandra_issues_analyzed = []
for each in list_issue_number:
  issue = each.split()
  if len(issue) == 1:
    elemento = issue[0].replace('"', '')
    elemento = elemento.replace(")", "")
    elemento = elemento.replace(".", "")
    list_cassandra_issues_analyzed.append(elemento)

print(f'{len(list_cassandra_issues_analyzed)} commits com Id de issue: {list_cassandra_issues_analyzed}')

100 commits com Id de issue: ['CASSANDRA-16618', 'CASSANDRA-16571', 'CASSANDRA-16618', 'CASSANDRA-15985', 'CASSANDRA-16736', 'CASSANDRA-16804', 'CASSANDRA-16714', 'CASSANDRA-16695', 'CASSANDRA-16690', 'CASSANDRA-16654', 'CASSANDRA-16633', 'CASSANDRA-16628', 'CASSANDRA-16495', 'CASSANDRA-13853', 'CASSANDRA-15897', 'CASSANDRA-16478', 'CASSANDRA-16478', 'CASSANDRA-16478', 'CASSANDRA-16146', 'CASSANDRA-15789', 'CASSANDRA-16152', 'CASSANDRA-15907', 'CASSANDRA-15814', 'CASSANDRA-14993', 'CASSANDRA-15012', 'CASSANDRA-14660', 'CASSANDRA-14660', 'CASSANDRA-13797', 'CASSANDRA-11500', 'CASSANDRA-13004', 'CASSANDRA-13559', 'CASSANDRA-12847', 'CASSANDRA-13412', 'CASSANDRA-13412', 'CASSANDRA-13228', 'CASSANDRA-12835', 'CASSANDRA-13276', 'CASSANDRA-13276', 'CASSANDRA-13307', 'CASSANDRA-13038', 'CASSANDRA-13090', 'CASSANDRA-13090', 'CASSANDRA-13090', 'CASSANDRA-13046', 'CASSANDRA-12348', 'CASSANDRA-12868', 'CASSANDRA-11743', 'CASSANDRA-12642', 'CASSANDRA-12499', 'CASSANDRA-12499', 'CASSANDRA-11844', '

In [10]:
dict_message_commits_valido = {}

for k,v in dict_message_commits.items():
  for each in v:
    #print(k, each)
    for cassandra_issue in list_cassandra_issues_analyzed:
      if cassandra_issue in each:
        print(k, cassandra_issue, each)
        dict_message_commits_valido[k] = cassandra_issue
        break

8857d0448914c2b8bb3e4e8f5f49b1d369f53893 CASSANDRA-16618 Fix broken IntelliJ configuration Authored by Jacek Lewandowski; reviewed by Ekaterina Dimitrova and Michael Semb Wever for CASSANDRA-16618
b2cfe2e5d7662c4d5900483d1838c718740ea13c CASSANDRA-16571 Fix warning about multiple versions of ant detected when running ant test patch by Ekaterina Dimitrova; reviewed by Brandon Williams and Mick Semb Wever for CASSANDRA-16571
d3e174fa226275e6019847f92f86e6400008cca5 CASSANDRA-16618 Fix broken IntelliJ configuration Authored by Jacek Lewandowski; reviewed by Ekaterina Dimitrova and Michael Semb Wever for CASSANDRA-16618
d3ddb190eed11dda305bcae75b6db9e0b9275459 CASSANDRA-15985 Patch by Aleksandr Sorokoumov; reviewed by Ekaterina Dimitrova and Brandon Williams for CASSANDRA-15985
cb0e4386d8ac2d13e7f594ae3d6cacc0b4246855 CASSANDRA-16736 patch by Kamlesh Ghoradkar; reviewed by Ekaterina Dimitrova, Adam Holmberg, David Capwell, Justin Chu and Brandon Williamms for CASSANDRA-16736
6e0b084d65774d

In [11]:
for k, v in dict_message_commits_valido.items():
  print(k,v)

8857d0448914c2b8bb3e4e8f5f49b1d369f53893 CASSANDRA-16618
b2cfe2e5d7662c4d5900483d1838c718740ea13c CASSANDRA-16571
d3e174fa226275e6019847f92f86e6400008cca5 CASSANDRA-16618
d3ddb190eed11dda305bcae75b6db9e0b9275459 CASSANDRA-15985
cb0e4386d8ac2d13e7f594ae3d6cacc0b4246855 CASSANDRA-16736
6e0b084d65774d5a973687410a3ebbf64f55bf95 CASSANDRA-16804
f3c2d3a0f9e1867f6976fabec35e4e02f5289c37 CASSANDRA-16714
a0af091a5cbceeaa2b8f724b1790b61ef512b033 CASSANDRA-16695
a9f472c432fbe5c45662837f7d7ee578f59fd862 CASSANDRA-16690
fdabda1da849efdb2f0066a341b9cc24de7fd05c CASSANDRA-16654
26163bb12aac590289aab6deabc21fe86371e22a CASSANDRA-16633
bd270260c847e0e8d0825c417215df60e2625539 CASSANDRA-16633
5be6d7e6b1ecaaa74d394922afb57c37a132827d CASSANDRA-16633
8fd046f2557e4d50e80986b813bc904dce625e6a CASSANDRA-16628
3edacd632c2f2c360ca1d931c6b4f6fd7326511f CASSANDRA-16495
ec9b7b9d376e9c98cedb3ad4eae90311923bc7bd CASSANDRA-13853
efde6a76f9807eaa0af80b5e85aa350ac8709a81 CASSANDRA-15897
6524903c22407504d55f29410d2e7f7

In [12]:
len(dict_message_commits_valido)

149

## Critical Files found by ATDCodeAnalyzer

In [15]:
lista_arquivos_criticos = ['StorageService', 'ColumnFamilyStore', 'DatabaseDescriptor', 'CompactionManager', 'StorageProxy', 'SSTableReader', 'Config', 'CassandraDaemon', 'SelectStatement', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService']
len(lista_arquivos_criticos)

12

# Access Cassandra Jira Issues

In [16]:
'''
credentials.conf
[credentials]
username = ?
password = ?
'''
config = configparser.ConfigParser()
config.read('credentials.conf')

username = config['credentials']['username']
password = config['credentials']['password']

SERVIDOR_JIRA = 'https://issues.apache.org/jira'

try:
  # Create a JIRA client instance
  jira = JIRA(
      server=SERVIDOR_JIRA,  # Replace with the Cassandra JIRA server URL
      basic_auth=(username, password),  # Replace with your JIRA credentials
  )
  print(f'Servidor {SERVIDOR_JIRA} acessado com sucesso!')
except Exception as ex:
  print(f'Erro ao acessar o Jira: {str(ex)}')

Servidor https://issues.apache.org/jira acessado com sucesso!


## Sample 1

Exemplo de como usar a lib jira do Python

In [17]:
# Search for open issues in the Cassandra project
issues = jira.search_issues('project = CASSANDRA and status=resolved',maxResults=10)

for issue in issues:
    print(f'Key: {issue.key}, Summary: {issue.fields.summary}, Status: {issue.fields.status}')

Key: CASSANDRA-18834, Summary: Testing Pubsub/QBot, Status: Resolved
Key: CASSANDRA-18833, Summary: Testing Pubsub/QBot, Status: Resolved
Key: CASSANDRA-18832, Summary: Add 5.0-alpha1 to cassandra-dtest upgrade_manifest.py, Status: Resolved
Key: CASSANDRA-18829, Summary: Fix flaky test: org.apache.cassandra.index.sai.disk.SelectiveIntersectionTest.tracingIsCorrectlyReported, Status: Resolved
Key: CASSANDRA-18828, Summary: Fix dtest: TestTopology.test_resumable_decommission, Status: Resolved
Key: CASSANDRA-18826, Summary: BLOG - Cassandra Contributor July Meeting Replay, Status: Resolved
Key: CASSANDRA-18823, Summary: Cleanup behaviour during node decommission caused missing replica, Status: Resolved
Key: CASSANDRA-18821, Summary: org.apache.cassandra.distributed.test.guardrails.GuardrailDiskUsageTest failed with Authentication error on host /127.0.0.1:9042: Provided username cassandra and/or password are incorrect, Status: Resolved
Key: CASSANDRA-18818, Summary: Add cqlshrc.sample and 

In [18]:
issue_CASSANDRA_18647 = jira.issue("CASSANDRA-18647")
print(f'Key: {issue_CASSANDRA_18647.key}, Summary: {issue_CASSANDRA_18647.fields.summary}, Status: {issue_CASSANDRA_18647.fields.status}')
print(f'Date resolved: {issue_CASSANDRA_18647.fields.created}')
print(f'Type: {issue_CASSANDRA_18647.fields.issuetype}')
print(f'Priority: {issue_CASSANDRA_18647.fields.priority}')
print(f'Time spent: {issue_CASSANDRA_18647.fields.timespent} ')
print(f'Description: {issue_CASSANDRA_18647.fields.description}')

Key: CASSANDRA-18647, Summary: CASTing a float to decimal adds wrong digits, Status: Resolved
Date resolved: 2023-07-04T21:27:05.217+0000
Type: Bug
Priority: Normal
Time spent: 600 
Description: If I create a table with a *float* (32-bit) column, and cast it to the *decimal* type, the casting wrongly passes through the double (64-bit) type and picks up extra, wrong, digits. For example, if we have a column e of type "float", and run

INSERT INTO tbl (p, e) VALUES (1, 5.2)

SELECT CAST(e AS decimal) FROM tbl WHERE p=1

The result is the "decimal" value 5.199999809265137, with all those extra wrong digits. It would have been better to get back the decimal value 5.2, with only two significant digits.

It appears that this happens because Cassandra's implementation first converts the 32-bit float into a 64-bit double, and only then converts that - with all the silly extra digits it picked up in the first conversion - into a "decimal" value.

Contrast this with CAST(e AS text) which works c

## Get all issues in range of commits of Critical Files

In [19]:
print('Aguarde...')
t1 = datetime.datetime.now()
print(t1)

issues_by_date_2020_2021_1 = jira.search_issues('project=CASSANDRA and created>="2021-03-01"and created<="2021-06-25"',maxResults=1000)
print("2020/2021/1:", len(issues_by_date_2020_2021_1))

issues_by_date_2020_2021_2 = jira.search_issues('project=CASSANDRA and created>="2020-01-01"and created<"2020-03-01"',maxResults=1000)
print("2020/2021/2:", len(issues_by_date_2020_2021_2))

issues_by_date_2019_2020_1 = jira.search_issues('project=CASSANDRA and created>="2019-06-01"and created<"2020-01-01"',maxResults=1000)
print("2019/2020/1:",len(issues_by_date_2019_2020_1))

issues_by_date_2019_2020_2 = jira.search_issues('project=CASSANDRA and created>="2019-01-01"and created<"2019-06-01"',maxResults=1000)
print("2019/2020/2:",len(issues_by_date_2019_2020_2))

issues_by_date_2018_2019 = jira.search_issues('project=CASSANDRA and created>="2018-01-01"and created<"2019-01-01"',maxResults=1000)
print("2018/2019:",len(issues_by_date_2018_2019))

issues_by_date_2017_2018_1 = jira.search_issues('project=CASSANDRA and created>="2017-06-01"and created<"2018-01-01"',maxResults=1000)
print("2017/2018:",len(issues_by_date_2017_2018_1))

issues_by_date_2017_2018_2 = jira.search_issues('project=CASSANDRA and created>="2017-01-01"and created<"2017-06-01"',maxResults=1000)
print("2017-2:",len(issues_by_date_2017_2018_2))

issues_by_date_2016_2017_1_1 = jira.search_issues('project=CASSANDRA and created>="2016-09-01"and created<"2017-01-01"',maxResults=1000)
print("2016/2017/1:",len(issues_by_date_2016_2017_1_1))

issues_by_date_2016_2017_1_2 = jira.search_issues('project=CASSANDRA and created>="2016-06-01"and created<"2016-09-01"',maxResults=1000)
print("2016/2017/2:",len(issues_by_date_2016_2017_1_2))

issues_by_date_2016_2017_2 = jira.search_issues('project=CASSANDRA and created>="2016-01-01"and created<"2016-06-01"',maxResults=1000)
print("2016.1:",len(issues_by_date_2016_2017_2))

issues_by_date_2015_2016_1 = jira.search_issues('project=CASSANDRA and created>="2015-10-03"and created<"2016-01-01"',maxResults=1000)
print("2015/2016/1:", len(issues_by_date_2015_2016_1))

issues_by_date_2015_2016_2 = jira.search_issues('project=CASSANDRA and created>="2015-08-03"and created<"2015-10-03"',maxResults=1000)
print("2015/2016/2:", len(issues_by_date_2015_2016_2))


t2 = datetime.datetime.now()
print(t2)
print(f'Tempo da consulta: {t2-t1}')

Aguarde...
2023-09-12 21:26:34.920749
2020/2021/1: 292
2020/2021/2: 144
2019/2020/1: 317
2019/2020/2: 192
2018/2019: 799
2017/2018: 574
2017-2: 480
2016/2017/1: 501
2016/2017/2: 650
2016.1: 934
2015/2016/1: 518
2015/2016/2: 485
2023-09-12 21:27:55.478114
Tempo da consulta: 0:01:20.557365


In [20]:
issues_by_date_2015_2021 =  issues_by_date_2020_2021_1 + issues_by_date_2020_2021_2 + issues_by_date_2019_2020_1 + issues_by_date_2019_2020_2 + issues_by_date_2018_2019 + issues_by_date_2017_2018_1 + issues_by_date_2017_2018_2 + issues_by_date_2016_2017_1_1 + issues_by_date_2016_2017_1_2 + issues_by_date_2016_2017_2 + issues_by_date_2015_2016_1 + issues_by_date_2015_2016_2
len(issues_by_date_2015_2021)

5886

In [21]:
dict_issues_2015_2021 = {}

list_key, list_summary, list_status, list_description, list_priority, list_timeestimate, list_timespent, list_issuetype = list(), list(), list(), list(), list(), list(), list(), list()

i = 1
for issue in issues_by_date_2015_2021:
    print(f'{i}, Key: {issue.key}, Summary: {issue.fields.summary}, Status: {issue.fields.status}, Priority: {issue.fields.priority}, Timespent: {issue.fields.timespent}, Time estimate: {issue.fields.timeestimate}')
    list_key.append(issue.key)
    list_summary.append(issue.fields.summary)
    list_status.append(issue.fields.status)
    list_description.append(issue.fields.description)
    list_priority.append(issue.fields.priority)
    list_timespent.append(issue.fields.timespent)
    list_timeestimate.append(issue.fields.timeestimate)
    list_issuetype.append(issue.fields.issuetype)
    i += 1

dict_issues_2015_2021['key'] = list_key
dict_issues_2015_2021['type'] = list_issuetype
dict_issues_2015_2021['summary'] = list_summary
dict_issues_2015_2021['status'] = list_status
dict_issues_2015_2021['description'] = list_description
dict_issues_2015_2021['priority'] = list_priority
dict_issues_2015_2021['timespent'] = list_timespent
dict_issues_2015_2021['timeestimate'] = list_timeestimate

A saída de streaming foi truncada nas últimas 5000 linhas.
887, Key: CASSANDRA-15006, Summary: Possible java.nio.DirectByteBuffer leak, Status: Open, Priority: Normal, Timespent: None, Time estimate: None
888, Key: CASSANDRA-15005, Summary: Configurable whilelist for UDFs, Status: Open, Priority: Low, Timespent: None, Time estimate: None
889, Key: CASSANDRA-15004, Summary: Anti-compaction briefly corrupts sstable state for reads, Status: Resolved, Priority: Urgent, Timespent: None, Time estimate: None
890, Key: CASSANDRA-15002, Summary: Avoid leaking threads when anticompaction fails, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
891, Key: CASSANDRA-15001, Summary: counter not accurate, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
892, Key: CASSANDRA-15000, Summary: Invalidate counter caches on `nodetool import`, Status: Open, Priority: Normal, Timespent: None, Time estimate: None
893, Key: CASSANDRA-14999, Summary: Incorrect fallb

In [22]:
df_cassandra_issues_2015_2021 = pd.DataFrame(dict_issues_2015_2021)
df_cassandra_issues_2015_2021

,key,type,summary,status,description,priority,timespent,timeestimate
0,CASSANDRA-16763,Task,Create Cassandra documentation content for new...,Resolved,We need create the content (asciidoc) to rende...,High,16800.0,0.0
1,CASSANDRA-16762,Task,Create content and UI components for new website,Resolved,We need create the content (asciidoc) and UI c...,High,NaN,NaN
2,CASSANDRA-16761,Epic,New Cassandra Website and Documentation,Resolved,This epic captures the work associated with th...,High,NaN,NaN
3,CASSANDRA-16760,Improvement,JMXTimer exposes attributes in inconsistent ti...,Resolved,JMXTimer objects are constructed with a durati...,Normal,2400.0,0.0
4,CASSANDRA-16759,Bug,Avoid memoizing the wrong min cluster version ...,Resolved,CASSANDRA-16525 avoids trying to calculate the...,Normal,NaN,NaN
...,...,...,...,...,...,...,...,...
5881,CASSANDRA-9959,Bug,Expected bloom filter size should not be an int,Resolved,We cast the expected number of rows in scrub a...,Normal,NaN,NaN
5882,CASSANDRA-9958,Improvement,Add support for the IN operator in secondary i...,Resolved,On a table like:\n{code}\nCREATE Table t (a in...,Low,NaN,NaN
5883,CASSANDRA-9957,Bug,Unable to build Apache Cassandra Under Debian ...,Resolved,Trying to use the tool CCM ( Cassandra Cluster...,Normal,NaN,NaN
5884,CASSANDRA-9956,Bug,Stream failed during a rebuild,Resolved,In an attempt to rebuild a node of a datacente...,Normal,NaN,NaN


## Get all bug issues in range of commits of Critical Files

Obs: a biblioteca jira suporta no máximo 1000 resultados

In [23]:
print('Aguarde...')
t1 = datetime.datetime.now()
print(t1)

issues_by_date_and_bug_2020_2021 = jira.search_issues('project=CASSANDRA and created>="2020-01-01"and created<="2021-06-25" and issuetype=Bug',maxResults=1000)
print("2020/2021:", len(issues_by_date_and_bug_2020_2021))

issues_by_date_and_bug_2019_2020 = jira.search_issues('project=CASSANDRA and created>="2019-01-01"and created<"2020-01-21" and issuetype=Bug',maxResults=1000)
print("2019/2020:",len(issues_by_date_and_bug_2019_2020))

issues_by_date_and_bug_2018_2019 = jira.search_issues('project=CASSANDRA and created>="2018-01-01"and created<"2019-01-21" and issuetype=Bug',maxResults=1000)
print("2018/2019:",len(issues_by_date_and_bug_2018_2019))

issues_by_date_and_bug_2017_2018_1 = jira.search_issues('project=CASSANDRA and created>="2017-06-01"and created<"2018-01-01" and issuetype=Bug',maxResults=1000)
print("2017/2018:",len(issues_by_date_and_bug_2017_2018_1))

issues_by_date_and_bug_2017_2018_2 = jira.search_issues('project=CASSANDRA and created>="2017-01-01"and created<"2017-06-01" and issuetype=Bug',maxResults=1000)
print("2017-2:",len(issues_by_date_and_bug_2017_2018_2))

issues_by_date_and_bug_2016_2017_1 = jira.search_issues('project=CASSANDRA and created>="2016-06-01"and created<"2017-01-01" and issuetype=Bug',maxResults=1000)
print("2016/2017:",len(issues_by_date_and_bug_2016_2017_1))

issues_by_date_and_bug_2016_2017_2 = jira.search_issues('project=CASSANDRA and created>="2016-01-01"and created<"2016-06-01" and issuetype=Bug',maxResults=1000)
print("2016.1:",len(issues_by_date_and_bug_2016_2017_2))

issues_by_date_and_bug_2015_2016 = jira.search_issues('project=CASSANDRA and created>="2015-08-03"and created<"2016-01-01" and issuetype=Bug',maxResults=1000)
print("2015/2016:", len(issues_by_date_and_bug_2015_2016))

t2 = datetime.datetime.now()
print(t2)
print(f'Tempo da consulta: {t2-t1}')

Aguarde...
2023-09-12 21:27:58.173565
2020/2021: 737
2019/2020: 346
2018/2019: 481
2017/2018: 374
2017-2: 321
2016/2017: 590
2016.1: 506
2015/2016: 510
2023-09-12 21:28:49.925662
Tempo da consulta: 0:00:51.752097


## Convert bug issues to Dataframe

Concatena todos os issues entre 2015 e 2021 do Jira/Cassandra

3.0.0(2015-08-03) até 3.11.11(2021-06-25)

In [24]:
issues_by_date_and_bug = issues_by_date_and_bug_2020_2021 + issues_by_date_and_bug_2019_2020 + issues_by_date_and_bug_2018_2019 + issues_by_date_and_bug_2017_2018_1 + issues_by_date_and_bug_2017_2018_2 + issues_by_date_and_bug_2016_2017_1 + issues_by_date_and_bug_2016_2017_2 + issues_by_date_and_bug_2015_2016

dict_issues_by_bug = {}

list_key, list_summary, list_status, list_description, list_priority, list_timeestimate, list_timespent, list_issuetype = list(), list(), list(), list(), list(), list(), list(), list()

i = 1
for issue in issues_by_date_and_bug:
    print(f'{i}, Key: {issue.key}, Summary: {issue.fields.summary}, Type: {issue.fields.issuetype}, Status: {issue.fields.status}, Priority: {issue.fields.priority}, Timespent: {issue.fields.timespent}, Time estimate: {issue.fields.timeestimate}')
    list_key.append(issue.key)
    list_summary.append(issue.fields.summary)
    list_status.append(issue.fields.status)
    list_description.append(issue.fields.description)
    list_priority.append(issue.fields.priority)
    list_timespent.append(issue.fields.timespent)
    list_timeestimate.append(issue.fields.timeestimate)
    list_issuetype.append(issue.fields.issuetype)
    i += 1

dict_issues_by_bug['key'] = list_key
dict_issues_by_bug['type'] = list_issuetype
dict_issues_by_bug['summary'] = list_summary
dict_issues_by_bug['status'] = list_status
dict_issues_by_bug['description'] = list_description
dict_issues_by_bug['priority'] = list_priority
dict_issues_by_bug['timespent'] = list_timespent
dict_issues_by_bug['timeestimate'] = list_timeestimate

1, Key: CASSANDRA-16759, Summary: Avoid memoizing the wrong min cluster version during upgrades, Type: Bug, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
2, Key: CASSANDRA-16758, Summary: Flaky ClientResourceLimitsTest, Type: Bug, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
3, Key: CASSANDRA-16757, Summary: Fix org.apache.cassandra.distributed.upgrade.CompactStorage2to3UpgradeTest, Type: Bug, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
4, Key: CASSANDRA-16756, Summary: Fix test testNullClusteringValues - org.apache.cassandra.distributed.upgrade.CompactStorage3to4UpgradeTest, Type: Bug, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
5, Key: CASSANDRA-16755, Summary: Spring boot application and AWS Cassandra DB connectivity issue, Type: Bug, Status: Resolved, Priority: Normal, Timespent: None, Time estimate: None
6, Key: CASSANDRA-16754, Summary: Flaky o.a.c.distributed.test.Sch

In [25]:
df_cassandra_bugs_2015_2021 = pd.DataFrame(dict_issues_by_bug)
df_cassandra_bugs_2015_2021

,key,type,summary,status,description,priority,timespent,timeestimate
0,CASSANDRA-16759,Bug,Avoid memoizing the wrong min cluster version ...,Resolved,CASSANDRA-16525 avoids trying to calculate the...,Normal,NaN,NaN
1,CASSANDRA-16758,Bug,Flaky ClientResourceLimitsTest,Resolved,Flaky [ClientResourceLimitsTest|https://ci-cas...,Normal,NaN,NaN
2,CASSANDRA-16757,Bug,Fix org.apache.cassandra.distributed.upgrade.C...,Resolved,{color:#172b4d}[https://jenkins-cm4.apache.org...,Normal,NaN,NaN
3,CASSANDRA-16756,Bug,Fix test testNullClusteringValues - org.apache...,Resolved,https://app.circleci.com/pipelines/github/dcap...,Normal,NaN,NaN
4,CASSANDRA-16755,Bug,Spring boot application and AWS Cassandra DB c...,Resolved,"Hi All,\r\n\r\nFacing an issue while starting ...",Normal,NaN,NaN
...,...,...,...,...,...,...,...,...
3860,CASSANDRA-9960,Bug,UDTs still visible after drop/recreate keyspace,Resolved,When deploying my app from the scratch I run s...,Low,NaN,NaN
3861,CASSANDRA-9959,Bug,Expected bloom filter size should not be an int,Resolved,We cast the expected number of rows in scrub a...,Normal,NaN,NaN
3862,CASSANDRA-9957,Bug,Unable to build Apache Cassandra Under Debian ...,Resolved,Trying to use the tool CCM ( Cassandra Cluster...,Normal,NaN,NaN
3863,CASSANDRA-9956,Bug,Stream failed during a rebuild,Resolved,In an attempt to rebuild a node of a datacente...,Normal,NaN,NaN


In [26]:
list_issue_bug_fix_2015_2021 = df_cassandra_bugs_2015_2021.key.to_list()
print('Lista de ID de Issue Bug entre 2015 e 2021')
print(f'{len(list_issue_bug_fix_2015_2021)}, {list_issue_bug_fix_2015_2021}')

Lista de ID de Issue Bug entre 2015 e 2021
3865, ['CASSANDRA-16759', 'CASSANDRA-16758', 'CASSANDRA-16757', 'CASSANDRA-16756', 'CASSANDRA-16755', 'CASSANDRA-16754', 'CASSANDRA-16753', 'CASSANDRA-16752', 'CASSANDRA-16751', 'CASSANDRA-16750', 'CASSANDRA-16748', 'CASSANDRA-16737', 'CASSANDRA-16735', 'CASSANDRA-16733', 'CASSANDRA-16732', 'CASSANDRA-16729', 'CASSANDRA-16728', 'CASSANDRA-16727', 'CASSANDRA-16726', 'CASSANDRA-16724', 'CASSANDRA-16723', 'CASSANDRA-16722', 'CASSANDRA-16721', 'CASSANDRA-16720', 'CASSANDRA-16718', 'CASSANDRA-16716', 'CASSANDRA-16714', 'CASSANDRA-16713', 'CASSANDRA-16712', 'CASSANDRA-16711', 'CASSANDRA-16710', 'CASSANDRA-16708', 'CASSANDRA-16707', 'CASSANDRA-16705', 'CASSANDRA-16704', 'CASSANDRA-16703', 'CASSANDRA-16702', 'CASSANDRA-16700', 'CASSANDRA-16699', 'CASSANDRA-16698', 'CASSANDRA-16694', 'CASSANDRA-16693', 'CASSANDRA-16692', 'CASSANDRA-16691', 'CASSANDRA-16690', 'CASSANDRA-16689', 'CASSANDRA-16688', 'CASSANDRA-16686', 'CASSANDRA-16685', 'CASSANDRA-16684', 

In [28]:
list_commits_bug_fix = []

for k, v in dict_message_commits_valido.items():
  if v in list_issue_bug_fix_2015_2021:
    print(k,v)
    list_commits_bug_fix.append(k)

8857d0448914c2b8bb3e4e8f5f49b1d369f53893 CASSANDRA-16618
b2cfe2e5d7662c4d5900483d1838c718740ea13c CASSANDRA-16571
d3e174fa226275e6019847f92f86e6400008cca5 CASSANDRA-16618
d3ddb190eed11dda305bcae75b6db9e0b9275459 CASSANDRA-15985
f3c2d3a0f9e1867f6976fabec35e4e02f5289c37 CASSANDRA-16714
a9f472c432fbe5c45662837f7d7ee578f59fd862 CASSANDRA-16690
fdabda1da849efdb2f0066a341b9cc24de7fd05c CASSANDRA-16654
26163bb12aac590289aab6deabc21fe86371e22a CASSANDRA-16633
bd270260c847e0e8d0825c417215df60e2625539 CASSANDRA-16633
5be6d7e6b1ecaaa74d394922afb57c37a132827d CASSANDRA-16633
8fd046f2557e4d50e80986b813bc904dce625e6a CASSANDRA-16628
3edacd632c2f2c360ca1d931c6b4f6fd7326511f CASSANDRA-16495
efde6a76f9807eaa0af80b5e85aa350ac8709a81 CASSANDRA-15897
6524903c22407504d55f29410d2e7f709104b8fd CASSANDRA-16478
8441d88d6dd3458d5520c619e9dc559d8ad6347b CASSANDRA-16478
c775ea3fa77bb661f405d8ebba738546518ac18e CASSANDRA-16478
e5ab8c1951384b9ddf0df9f1d4d49b4c9dfc188f CASSANDRA-16146
fa9bbd431100ceac0af8ca3ea0a3dac

In [29]:
len(list_commits_bug_fix)

85

In [30]:
print(list_commits_bug_fix)

['8857d0448914c2b8bb3e4e8f5f49b1d369f53893', 'b2cfe2e5d7662c4d5900483d1838c718740ea13c', 'd3e174fa226275e6019847f92f86e6400008cca5', 'd3ddb190eed11dda305bcae75b6db9e0b9275459', 'f3c2d3a0f9e1867f6976fabec35e4e02f5289c37', 'a9f472c432fbe5c45662837f7d7ee578f59fd862', 'fdabda1da849efdb2f0066a341b9cc24de7fd05c', '26163bb12aac590289aab6deabc21fe86371e22a', 'bd270260c847e0e8d0825c417215df60e2625539', '5be6d7e6b1ecaaa74d394922afb57c37a132827d', '8fd046f2557e4d50e80986b813bc904dce625e6a', '3edacd632c2f2c360ca1d931c6b4f6fd7326511f', 'efde6a76f9807eaa0af80b5e85aa350ac8709a81', '6524903c22407504d55f29410d2e7f709104b8fd', '8441d88d6dd3458d5520c619e9dc559d8ad6347b', 'c775ea3fa77bb661f405d8ebba738546518ac18e', 'e5ab8c1951384b9ddf0df9f1d4d49b4c9dfc188f', 'fa9bbd431100ceac0af8ca3ea0a3dac407246446', 'fee7a10823da1e29bd0e6504fea9679389180c9e', '9f8d5b8d069a1db88e70deafff6c0edc23c896d0', '4d42c189fa82b32fd93ae42a164b91e4db62992e', 'a4af4aa60ad0aa6fbb134b44e162970c7a1a20da', 'c94a6aa7e5dd6d46f99dd07bec1645

In [46]:
dict_commit_modified_files_impacted_by_bugs = {}
list_modified_files_impacted_by_bugs = []

for commit in pydriller.Repository("cassandra", only_commits=list_commits_bug_fix).traverse_commits():
    list_modified_files = []
    for m in commit.modified_files:
        list_modified_files.append(m.filename)
        list_modified_files_impacted_by_bugs.append(m.filename)
    dict_commit_modified_files_impacted_by_bugs[commit.hash] = list_modified_files
    print(commit.hash, list_modified_files)

ebbd516985bc3e2859ae00e63a024b837cb4b429 ['DateType.java', 'TimestampType.java', 'TimestampSerializer.java']
e43883137750d62d8bafb3058d5ee4e322221733 ['CQL.textile']
4e5867affc6c02b4a84ecee5e82e2ac15c2e3a1c ['CQL.textile', 'QueryProcessor.java']
f8452838a964c7bfa938277c9fdfc337f9aff886 ['CHANGES.txt', 'Cql.g']
9dafa438a5dcce8674aaa945b1495e70f95d2839 ['CHANGES.txt', 'StartupChecks.java']
df3d0b00b0d9a0725b3f1681e6ce9ffe6d330de4 ['CHANGES.txt', 'MetadataCollector.java', 'TTLExpiryTest.java']
bc3ea66925429b743b672d417700d17e9936b187 ['DeleteTest.java']
58821cefb2cc2b1bed0d28533dddea0323ecf487 ['DeleteTest.java']
093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoa

In [36]:
set_list_modified_files_impacted_by_bugs = set(list_modified_files_impacted_by_bugs)
print(f'{len(set_list_modified_files_impacted_by_bugs)}, {set_list_modified_files_impacted_by_bugs}')

154, {'StreamingHistogramBench.java', 'ApplicationState.java', 'MigrationManager.java', 'CompactionManager.java', 'TableViews.java', 'VersionedValue.java', 'SSTableRewriterTest.java', 'RowCacheTest.java', 'CQL.textile', 'SnapshotVerbHandler.java', 'MigrationCoordinator.java', 'StreamingHistogramTest.java', 'DuplicateRowCheckerTest.java', 'TimeWindowCompactionStrategyTest.java', 'MixedModeReadRepairTest.java', 'VersionAndType.java', 'QueryMessage.java', 'LongLeveledCompactionStrategyTest.java', 'SASIIndex.java', 'ColumnFilter.java', 'ReadCommand.java', 'ReadResponse.java', 'SchemaLoader.java', 'AbstractBTreePartition.java', 'SSTable.java', 'Config.java', 'Lists.java', 'LeveledCompactionStrategyTest.java', 'FilteredPartitions.java', 'AlterViewStatement.java', 'wcwidth.py', 'FBUtilitiesTest.java', 'CompactionIterator.java', 'SinglePartitionReadCommand.java', 'DuplicateRowChecker.java', 'StartupChecks.java', 'CFMetaData.java', 'ViewComplexTest.java', 'copyutil.py', 'FrozenCollectionsTest.j

In [40]:
lista_arquivos_criticos_java = []
for each in lista_arquivos_criticos:
  each = each + '.java'
  lista_arquivos_criticos_java.append(each)
lista_arquivos_criticos_java

['StorageService.java',
 'ColumnFamilyStore.java',
 'DatabaseDescriptor.java',
 'CompactionManager.java',
 'StorageProxy.java',
 'SSTableReader.java',
 'Config.java',
 'CassandraDaemon.java',
 'SelectStatement.java',
 'SinglePartitionReadCommand.java',
 'NodeProbe.java',
 'MessagingService.java']

In [41]:
conjunto_lista_arquivos_criticos = set(lista_arquivos_criticos_java)

### Arquivos críticos que aparecem nos bug-fix

In [43]:
classes_criticas_aparece_em_bugfix = conjunto_lista_arquivos_criticos.intersection(set_list_modified_files_impacted_by_bugs)
classes_criticas_aparece_em_bugfix

{'ColumnFamilyStore.java',
 'CompactionManager.java',
 'Config.java',
 'DatabaseDescriptor.java',
 'MessagingService.java',
 'SinglePartitionReadCommand.java',
 'StorageProxy.java',
 'StorageService.java'}

In [54]:
my_commits_bugfix = []
for k, v in dict_commit_modified_files_impacted_by_bugs.items():
  for each in classes_criticas_aparece_em_bugfix:
    if each in v:
      print(k,v)
      my_commits_bugfix.append(k)

093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
093454205498bad7094cb8fb362d60e95296b092 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
621f446ca4a319adcb50d2479e3fa7121eac6ebd ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
965ee082da2b8a3ff5e932e91d261018d9232c75 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
2d420e6c2a362c69d1a66076ac7dd2db37ee9306 ['CHANGES.txt', 'Config.java', 'DatabaseDescriptor.java', 'YamlConfigurationLoader.java']
2d420e6c2a362c69d1a66076ac7dd2db37ee9306 ['CHANGES.txt', 'Config.java', 'DatabaseDe

In [55]:
my_commits_bugfix

['093454205498bad7094cb8fb362d60e95296b092',
 '093454205498bad7094cb8fb362d60e95296b092',
 '621f446ca4a319adcb50d2479e3fa7121eac6ebd',
 '621f446ca4a319adcb50d2479e3fa7121eac6ebd',
 '965ee082da2b8a3ff5e932e91d261018d9232c75',
 '965ee082da2b8a3ff5e932e91d261018d9232c75',
 '2d420e6c2a362c69d1a66076ac7dd2db37ee9306',
 '2d420e6c2a362c69d1a66076ac7dd2db37ee9306',
 'c947a2c4228816f0e069b2f6fed4b725c9cb710e',
 'c947a2c4228816f0e069b2f6fed4b725c9cb710e',
 'd200d137823d5b250406bccb35473a8fc2f14faf',
 'ec604875886a5790b627f0fa22cea4fca34532dd',
 'eace9aaddfdd0059f52b1eb9b6902f999f04a447',
 '5725e2c422d21d8efe5ae3bc4389842939553650',
 '5725e2c422d21d8efe5ae3bc4389842939553650',
 '1f54aa424fd8a79089f76951a93560e6bca9d459',
 '90c5c8bfecd69edb5f859e9ff2424930b1394b49',
 '4d42c189fa82b32fd93ae42a164b91e4db62992e',
 '4d42c189fa82b32fd93ae42a164b91e4db62992e',
 '4d42c189fa82b32fd93ae42a164b91e4db62992e',
 'fee7a10823da1e29bd0e6504fea9679389180c9e',
 'e5ab8c1951384b9ddf0df9f1d4d49b4c9dfc188f',
 'efde6a76

In [59]:
lista_mensagens_temp = []
for each_commit in my_commits_bugfix:
  os.environ['COMMIT_BUG_FIX'] = each_commit
  mensagem_commit = !cd cassandra && git log --format=%B -n 1 $COMMIT_BUG_FIX
  elemento = each_commit, mensagem_commit
  lista_mensagens_temp.append(elemento)

In [60]:
lista_mensagens_temp

[('093454205498bad7094cb8fb362d60e95296b092',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217',
   '']),
 ('093454205498bad7094cb8fb362d60e95296b092',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217',
   '']),
 ('621f446ca4a319adcb50d2479e3fa7121eac6ebd',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217',
   '']),
 ('621f446ca4a319adcb50d2479e3fa7121eac6ebd',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217',
   '']),
 ('965ee082da2b8a3ff5e932e91d261018d9232c75',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown, reviewed by Paulo Motta for CASSANDRA-11217',
   '']),
 ('965ee082da2b8a3ff5e932e91d261018d9232c75',
  ['Only log yaml config once, at startup',
   '',
   'patch by jasobrown,